In [8]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#import libraries

import numpy as np
import tensorflow.keras as keras
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
# CONSTANTS

PATH_TO_DRIVE = "/content/drive/My Drive"
DATASET_PATH = PATH_TO_DRIVE + "/datasets/noise-speech-data-processed.json"
REDUCED_DATASET_PATH = PATH_TO_DRIVE + "/datasets/noise-speech-data-reduced-processed.json"

In [5]:
# Plot training history of model

def plot_history(history):
  fig, axs = plt.subplots(2)

  # Create accuracy subplot
  axs[0].plot(history.history["accuracy"], label="train accuracy")
  axs[0].plot(history.history["val_accuracy"], label="val accuracy")
  axs[0].set_ylabel("Accuracy")
  axs[0].legend(loc="lower rights")
  axs[0].set_title("Accuracy Eval")

  # Create error subplot
  axs[1].plot(history.history["loss"], label="train error")
  axs[1].plot(history.history["val_loss"], label="val error")
  axs[1].set_ylabel("Error")
  axs[1].set_xlabel("Epoch")
  axs[1].legend(loc="lower rights")
  axs[1].set_title("Error Eval")

  plt.show()

In [4]:
#Function to load processed audio data from JSON

def load_data(dataset_path):
  with open(dataset_path, "r") as fp:
    data = json.load(fp)

  # Convet list to numpy arrays
  inputs = np.array(data["mfcc"])
  targets = np.array(data["labels"])

  return inputs, targets

In [21]:
def prepare_datasets(test_size, validation_size):

  # Load data
  X, y = load_data(DATASET_PATH)

  # create train-test split
  X_train, X_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      test_size=test_size)
  
  # create train-validation split
  X_train, X_validation, y_train, y_validation = train_test_split(X_train,
                                                                  y_train,
                                                                  test_size=validation_size)
  
  # Add new dimension to data
  X_train = X_train[..., np.newaxis] # 4d array -> (num_samples, 216, 13, 1)
  X_test = X_test[..., np.newaxis] # 4d array -> (num_samples, 216, 13, 1)
  X_validation = X_validation[..., np.newaxis] # 4d array -> (num_samples, 216, 13, 1)

  return X_train, X_validation, X_test, y_train, y_validation, y_test

In [60]:
# Function to build model

def build_model(input_shape):
  
  # create model
  model = keras.Sequential()

  # 1st conv layer
  model.add(keras.layers.Conv2D(4,
                                (2,2),
                                activation="relu",
                                input_shape=input_shape))
  model.add(keras.layers.MaxPool2D((2,2),
                                   strides=(1,1),
                                   padding="same"))
  model.add(keras.layers.BatchNormalization())

  # # 2nd conv layer
  # model.add(keras.layers.Conv2D(32,
  #                               (3,3),
  #                               activation="relu",
  #                               input_shape=input_shape))
  # model.add(keras.layers.MaxPool2D((3,3),
  #                                  strides=(2,2),
  #                                  padding="same"))
  # model.add(keras.layers.BatchNormalization())

  # # 3rd conv layer
  # model.add(keras.layers.Conv2D(32,
  #                               (2,2),
  #                               activation="relu",
  #                               input_shape=input_shape))
  # model.add(keras.layers.MaxPool2D((2,2),
  #                                  strides=(2,2),
  #                                  padding="same"))
  # model.add(keras.layers.BatchNormalization())
  
  # Flatten the output and feed it to a dense layer
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(2, activation="relu"))
  # model.add(keras.layers.Dropout(0.3))

  # Output layer
  model.add(keras.layers.Dense(2, activation="softmax"))

  return model

In [42]:
# Build the train, test and validation sets
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25,0.2)
print(X_train.shape)

(1200, 216, 13, 1)


In [61]:
# extract input shape from input sets
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

# Build the CNN model
model = build_model(input_shape)

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 215, 12, 4)        20        
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 215, 12, 4)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 215, 12, 4)        16        
_________________________________________________________________
flatten_8 (Flatten)          (None, 10320)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 20642     
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 6         
Total params: 20,684
Trainable params: 20,676
Non-trainable params: 8
__________________________________________________

In [65]:
# Add optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.0002)

# Compile the network
model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [67]:
# Train the CNN
model.fit(X_train, y_train,
          validation_data=(X_validation, y_validation),
          batch_size=5,
          epochs=20)

Epoch 1/20
240/240 [==============================] - 3s 11ms/step - loss: 0.0433 - accuracy: 0.9992 - val_loss: 0.0387 - val_accuracy: 1.0000
Epoch 2/20
240/240 [==============================] - 3s 11ms/step - loss: 0.0397 - accuracy: 1.0000 - val_loss: 0.0348 - val_accuracy: 1.0000
Epoch 3/20
240/240 [==============================] - 3s 11ms/step - loss: 0.0373 - accuracy: 1.0000 - val_loss: 0.0324 - val_accuracy: 1.0000
Epoch 4/20
240/240 [==============================] - 3s 11ms/step - loss: 0.0345 - accuracy: 1.0000 - val_loss: 0.0304 - val_accuracy: 1.0000
Epoch 5/20
240/240 [==============================] - 3s 11ms/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 0.0288 - val_accuracy: 1.0000
Epoch 6/20
240/240 [==============================] - 3s 11ms/step - loss: 0.0300 - accuracy: 1.0000 - val_loss: 0.0266 - val_accuracy: 1.0000
Epoch 7/20
240/240 [==============================] - 3s 11ms/step - loss: 0.0281 - accuracy: 1.0000 - val_loss: 0.0254 - val_accuracy: 1.0000

In [68]:
# Evaluate the CNN on test set
test_error, test_accuracy = model.evaluate(X_test,
                                           y_test,
                                           verbose=1)

print("Accuracy on test set is : {}".format(test_accuracy))

16/16 [==============================] - 0s 14ms/step - loss: 0.0232 - accuracy: 0.9940
Accuracy on test set is : 0.9940000176429749
